In [101]:
import sys
sys.path.append("../model/")
import os
import unittest

## 1. test ops.py

In [2]:
from ops import InstanceNormalization, PermaDropout, conv_bn_activation
from keras.models import Model
from keras.layers import Input
import numpy as np

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [161]:
inputs = Input(shape=(1000,))
outputs = PermaDropout(0.1)(inputs)
model = Model(inputs=inputs,outputs=outputs)

test_input = np.expand_dims(np.arange(1000), axis=0)
test_output = model.predict_on_batch(test_input)

np.sum(test_output == 0) <= 120

True

In [98]:
class TestInstanceNormalization(unittest.TestCase):
    def test_predict_norm_output_with_vector_input(self):
        '''
        input  : [   0,     1,       2,      3,     4,      5,     6,     7,     8  ]
        output : [-1.549, -1.161, -0.774, -0.387,   0,  0.387, 0.775, 1.162,   1.549]
        '''
        # setup Model
        inputs = Input(shape=(9,))
        outputs = InstanceNormalization()(inputs)
        model = Model(inputs=inputs,outputs=outputs)
        
        test_input = np.expand_dims(np.arange(9), axis=0)
        test_answer = (test_input - np.mean(test_input)) / np.std(test_input)
        test_output = model.predict_on_batch(test_input)
        MSE =  np.sqrt(np.mean(np.square(test_answer - test_output)))
        self.assertLessEqual(MSE, 0.001)
        
    def test_train_norm_output_with_vector_input_no_scale_and_no_center(self):
        '''
        input  : [   0,     1,       2,      3,     4,      5,     6,     7,     8  ]
        output : [-1.549, -1.161, -0.774, -0.387,   0,  0.387, 0.775, 1.162,   1.549]
        
        scale=False, center=False 하면,
        instance Normalization에서 기억하는 값이 없으므로 training하더라도 같은 Loss 값이 계속 나오게 되어야 한다.
        '''
        inputs = Input(shape=(9,))
        outputs = InstanceNormalization(scale=False, center=False)(inputs)
        model = Model(inputs=inputs,outputs=outputs)
        model.compile('sgd','mae')
        
        test_input = np.expand_dims(np.arange(9), axis=0)
        test_answer = (test_input - np.mean(test_input)) / np.std(test_input)

        MAE = model.train_on_batch(test_input,test_answer)
        self.assertLessEqual(MAE, 0.0005)
        for _ in range(2):
            self.assertEqual(MAE, model.train_on_batch(test_input,test_answer))
    
    def test_predict_norm_output_with_2Dmatrix_input(self):
        '''
        input : [[0,1,2],
                 [3,4,5],
                 [6,7,8]]
        output : [[-1.549, -1.161, -0.774],
                  [-0.387,      0,  0.387],
                  [0.775,   1.162,  1.549]]
        '''
        inputs = Input(shape=(3,3))
        outputs = InstanceNormalization()(inputs)
        model = Model(inputs=inputs,outputs=outputs)

        test_input = np.reshape(np.arange(9),(3,3))
        test_input = np.expand_dims(test_input, axis=0)
        test_answer = (test_input - test_input.mean()) / test_input.std()
        test_output = model.predict_on_batch(test_input)
        MSE =  np.sqrt(np.mean(np.square(test_answer - test_output)))
        self.assertLessEqual(MSE, 0.001)

In [99]:
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
...
----------------------------------------------------------------------
Ran 3 tests in 0.386s

OK
